# 작사가 인공지능 만들기

---
프로젝트에 들어가기전에 GPT-2 모델로 구현된 인공지능 작문가의 수준을 체험해보도록하자
- 요리 레시피나 SF 소설도 써주는 인공지능 작문가 : https://talktotransformer.com/

이번프로젝트를 통해 위의 사례처럼  인공지능이 문장을 이해하는 방식을 알아보고 작문을 학습시켜 작사가 인공지능을 만들어보자

프로젝트 목차 

1. 시퀀스? 스퀀스!
2. I 다음 am을 쓰면 반 이상은 맞더라
3. 실습
 - 데이터 다듬기
 - 인공지능 학습시키기
 - 잘 만들어졌는지 평가하기
4. 프로젝트 : 멋진 작사가 만들기

# 시퀀스 데이터란?
- 시퀀스 데이터는 나열된 데이터를 의미한다
- 사례
 - 텍스트데이터 : 신문기사, 시, 소설
 - 시계열 수치 데이터 : 월별 상품 판매량 변화, 일별 주식 시황 데이터
- 각 요소들이 동일한 속성을 띌 필요는 없지만 **통계에 기반하여** 추측이 가능할 정도의 연관성은 있어야한다.


## RNN(Recurrent Neural Network)이란?
![이미지](https://stanford.edu/~shervine/teaching/cs-230/illustrations/architecture-rnn-ltr.png?9ea4417fc145b9346a3e288801dbdfdc)

좌측에서 우측으로 문장의 입력을  x<1> 부터 신경망에 입력해준다. 그러면 신경망에서 출력y<1>을 예측한다. 그리고 문장의 두번째 단어 x<2> 를 읽고 이번에는 첫번재 단어에서 연산한 정보(활성화되어 나온값)를 사용하여 출력y<2>를 예측한다. 이렇게 이전셀의 정보가 순환적으로 연결되어 마지막 출력값을 도출하는 구조가 RNN모델이다. 즉 x<1>, x<2> ..x<n-1>이 주어졌을때 n번째 단어 y<n>을 확률적으로예측하는 구조이다.

신경망 학습을 시작하기 전에 a<0>가 사용되는 것을 볼 수 있는데, 이것은 보통 무작위로 초기화하거나, 0으로 초기화한다


# 데이터셋 준비
이번 프로젝트에서는 연극대사를 만들어내는 모델 구현할 것이므로 연극대본 파일을 준비한 후 모델의 입력으로 사용할수 있도록 전처리까지 해보자


### 파일 읽기

In [51]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/project/exp04_LyricistAI/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:11])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '', 'All:', 'Resolved. resolved.']


### 데이터 전처리1
연극대사를 만들기위한 데이터가 필요하므로 읽어드린 파일 내용에 전처리가 필요하다.
필터링할 데이터
- 화자이름
- 공백대사
- 문장끝에 : 기호 제외


In [52]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 데이터 전처리2
필터링하여 깔끔한 문장이 출력되었다. 이제부터 우리가 구현할 모델에게 학습시킬 단어사전을 준비해야한다. 전처리한 문장을 학습에 맞게 쪼개는 토큰화작업이 필요하다

토큰화는 보통 띄어쓰기, 즉, 문장을 공백을 기준으로 나누는데 그렇게 했을경우 생기는 예외적인 단어들을 처리해줄 필요가 있다


- Hi, my name is John. *("Hi," "my", ..., "john." 으로 분리됨) - 문장부호
- First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자
- He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자


문장의 토큰화
 1. 소문자로 바꾸고, 양쪽 공백을 지운다
 2. 특수문자 양쪽에 공백을 넣는다
 3. 여러개의 공백은 하나의 공백으로 바꾼다
 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다
 5. 다시 양쪽 공백을 지운다
 6. 문장 시작에는 <start>, 끝에는 <end>를 추가

In [53]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 데이터셋 만들기

소스 문장(Source Sentence) : 자연어 처리에서 모델의 입력이 되는 문장
 - <start> this is sample sentence . 
 -  X_train
    
타겟 문장(Target Sentence) : 정답 역할을 하게 될 모델의 출력 문장
 - this is sample sentence . <end>
 - y_train

In [54]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence)==0 : continue
    if sentence[-1]==':':continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### 데이터의 벡터화
이미지를 학습시킬때 픽셀정보가 수치화되어 있듯이 우리가 준비한 데이터셋도 수치화해주는 작업이 필요하다. 여기서는 텐서플로우의 tf.keras.preprocessing.text.Tokenizer 라이브러리를 사용하여 데이터를 토큰화하여 단어사전을 만들고 데이터를 숫자로 변경해주는 벡터화를 진행할 것이다. 숫자로 변환된 데이터를 텐서(tensor) 라고 칭한다고 한다.

- [텐서플로우](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)
- 텐서 개념 : [참고](https://rekt77.tistory.com/102)

In [55]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer


tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f60f66fd640>


텐서 데이터로 변환되었다. 이숫자들은  tokenizer에 구축된 단어 사전의 인덱스이다.

In [24]:
len(tensor), tensor[0], tensor[1]

(24015,
 array([  2, 143,  40, 933, 140, 591,   4, 124,  24, 110,   5,   3,   0,
          0,   0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([  2, 110,   4, 110,   5,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0], dtype=int32))

24015의 문장이 텐서로 변환되었다. tensor[0]를 보면 첫번째 문장 \<start> before we proceed any further , hear me speak . \<end> 의 각 단어가 숫자로 어떻게 매칭되어있는지 확인 할 수 있다
    
텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것이다. 0은 <pad> 문자가 된다
    
tokenizer에 구축된 단어 사전의 인덱스를 통해 단어 사전이 어떻게 구축되었는지 확인해보자

In [17]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    if idx>=10:break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


2번 인덱스가 바로 <start>이다. tensor[0], tensor[1] 문자 모두 2로 시작하는 이유가 바로 이때문이다

### 텐서를 소스와 타겟으로 분리

In [25]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


소스는 2(<start>)에서 시작해서 3(<end>)으로 끝난 후 0(<pad>)로 채워져 있다. 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 한 형태를 가지고 있다.
    
텐서플로우의 입력 데이터는 [tf.data.Dataset]( https://www.tensorflow.org/api_docs/python/tf/data/Dataset)객체가 필요하다. 이미 텐서형태로 만들어두었으므로 입력으로 설정해보자

In [26]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset


<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

데이터 전처리 과정이 아주 길었는데 정리해보면 다음과 같다

- 정규표현식을 이용한 corpus 생성
- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환
---

# 모델 학습

이프로젝트에서 만들어볼 모델의 구조는 대략적으로 아래와 같다
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성된다
![이미지](https://d3s0tskafalll9.cloudfront.net/media/images/E-12-4.max-800x600.png)

In [28]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

### 모델에 대한 간략한 이해
입력 데이터
- 단어 사전의 인덱스

Embedding 레이어
- 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다
- 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용된다
- embedding_size : 256, 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
 -  크기가 2라면 차갑다: [0.0, 1.0], 뜨겁다: [1.0, 0.0], 미지근하다: [0.5, 0.5] 이렇게 단어의 추상적 특징을 수치로 벡터공간에서 표현한다.

LSTM 레이어
- hidden_size : 1024, hidden state 의 차원수
- 데이터를 이해하는 신경망 노드의 수
- return_sequences=True : 자신에게 입력된 시퀀스 길이만큼 동일한 시퀀스를 출력
- return_sequences=False : 1개의 벡터만 출력

Dense 레이어
 - 출력값

### 하나의 배치를 모델에 넣어보기

In [29]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.86885634e-04, -1.48957100e-04, -5.80529049e-05, ...,
          1.01569116e-04, -4.68944882e-06,  2.13540683e-04],
        [ 4.98897971e-05, -9.70240653e-05,  3.48296780e-05, ...,
          3.05136182e-05, -6.40581493e-05,  7.12505600e-04],
        [-2.40737674e-04, -1.07917454e-04, -1.71810665e-04, ...,
          2.24614254e-04, -2.09224963e-04,  1.09316793e-03],
        ...,
        [ 1.50357140e-03, -9.37929610e-04, -1.79155776e-03, ...,
          9.64677543e-04, -1.74297439e-03, -7.50154897e-04],
        [ 1.80603168e-03, -1.02920923e-03, -1.98678765e-03, ...,
          1.01376523e-03, -1.68514310e-03, -9.61459649e-04],
        [ 2.12527975e-03, -1.10701856e-03, -2.16958974e-03, ...,
          1.05384516e-03, -1.57800457e-03, -1.12249458e-03]],

       [[ 1.86885634e-04, -1.48957100e-04, -5.80529049e-05, ...,
          1.01569116e-04, -4.68944882e-06,  2.13540683e-04],
        [ 1.40491786e-04, -3.93041933e-04, -1.

최종 출력텐서shape이  tf.Tensor: shape=(256, 20, 7001) 으로 확인된다.
- 256 : 배치사이즈 , 문장 개수
- 20 : 한 문장의 길이
- 7001 : Dense 레이어의 출력 차원수, 단어사전의 총단어수


In [30]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


모델이 빌드되는 시점에서는 입력 데이터의 시퀀스 길이가 얼마인지 모르기때문에 Output Shape 의 값이 multiple 로 표시되었다
모델의 파라미터 사이즈는 대략 22million이다. 보통 GPT-2의 파라미터 사이즈는, 1.5billion이다.

이제 모델을 학습시켜보자~ (GPU기준 15분정도 소요 예정)

In [31]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 37s 374ms/step - loss: 3.4880
Epoch 2/30
93/93 [==============================] - 29s 309ms/step - loss: 2.8047
Epoch 3/30
93/93 [==============================] - 36s 386ms/step - loss: 2.7013
Epoch 4/30
93/93 [==============================] - 36s 386ms/step - loss: 2.6018
Epoch 5/30
93/93 [==============================] - 37s 394ms/step - loss: 2.5327
Epoch 6/30
93/93 [==============================] - 37s 395ms/step - loss: 2.4741
Epoch 7/30
93/93 [==============================] - 37s 393ms/step - loss: 2.4153
Epoch 8/30
93/93 [==============================] - 37s 396ms/step - loss: 2.3602
Epoch 9/30
93/93 [==============================] - 36s 391ms/step - loss: 2.3093
Epoch 10/30
93/93 [==============================] - 33s 357ms/step - loss: 2.2583
Epoch 11/30
93/93 [==============================] - 37s 396ms/step - loss: 2.2082
Epoch 12/30
93/93 [==============================] - 37s 396ms/step - loss: 2.1606
Epoch 13/30
9

### Loss는... 
모델이 오답을 만들고 있는 정도, 어떤 학습된 모델을 실제 데이터에 적용했을 때 모델의 추정 오차로 인해 발생하는 손실값이다, 이번 학습에서 30epochs에 1.2644까지 안정적으로 주는 모습을 보여주고있다

---

# 모델 평가
학습한 모델이 어느정도 작문실력을 보여주는지 알아보는 함수를 만들어보자

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

테스트 단계에서 이전 프로젝트에서 진행했던 테스트셋이 따로 없다. \<start> 와 시작 단어 하나만 넣어주면된다. 그러면 모델이 while문을 돌면서 순차적으로 그 다음 단어를 만들어내는 것이다.

- while의 첫 번째 루프에서 test_tensor에 \<start> 하나만 들어갔다고 합시다. 우리의 모델이 출력으로 7001개의 단어 중 A를 고른다
- while의 두 번째 루프에서 test_tensor에는 \<start> A가 들어갑니다. 그래서 우리의 모델이 그다음 B를 고른다
- while의 세 번째 루프에서 test_tensor에는 \<start> A B가 들어가고 계속 이렇게 이어진다

In [50]:
generate_text(model, tokenizer, init_sentence="<start> she ")

'<start> she is not fourteen . <end> '

제법 멋진 문장을 만들어 내고있다. GPT-2, GPT-3모델을 쓴다면 더 똑똑한 문장을 만들어낼 것이다.